In [ ]:
#####################################################
############EJERCICIO2###############################
#PAQUETES PARA QUE FUNCIONE EL PROGRAMA
# pandas: Para manipulación de datos en DataFrames
# pip install pandas

# numpy: Para operaciones numéricas avanzadas
# pip install numpy

# python-docx: Para generar documentos de Word (.docx)
# pip install python-docx

# Pillow (PIL): Para procesamiento de imágenes
# pip install pillow


# requests: Para realizar solicitudes HTTP (ej. descargar imágenes)
# pip install requests

# RDKit: Para cálculos y visualización de estructuras químicas
# pip install rdkit (o conda install -c conda-forge rdkit)


In [19]:
import pandas as pd
import numpy as np
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.shared import OxmlElement, qn
import os
from datetime import datetime
import io
import base64
from PIL import Image
import requests
from rdkit import Chem
from rdkit.Chem import Draw, Descriptors, rdMolDescriptors, Crippen
import warnings
warnings.filterwarnings('ignore')

class BBBPReportGenerator:
    def __init__(self, csv_path):
        """
        Inicializa el generador de informes BBBP
        
        Args:
            csv_path (str): Ruta al archivo CSV con los datos BBBP
        """
        self.df = pd.read_csv(csv_path)
        self.output_dir = "informes_bbbp"
        self.create_output_directory()
        
    def create_output_directory(self):
        """Crea el directorio de salida si no existe"""
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)
    
    def add_black_heading(self, doc, text, level):
        """
        Añade un encabezado con color negro
        
        Args:
            doc: Documento de Word
            text (str): Texto del encabezado
            level (int): Nivel del encabezado
        
        Returns:
            Párrafo del encabezado creado
        """
        heading = doc.add_heading(text, level=level)
        # Cambiar color del texto a negro
        for run in heading.runs:
            run.font.color.rgb = RGBColor(0, 0, 0)  # RGB para negro
        return heading
            
    def calculate_molecular_properties(self, smiles):
        """
        Calcula propiedades moleculares usando RDKit
        
        Args:
            smiles (str): Código SMILES de la molécula
            
        Returns:
            dict: Diccionario con propiedades moleculares
        """
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                return None
                
            properties = {
                'molecular_weight': round(Descriptors.MolWt(mol), 2),
                'logp': round(Descriptors.MolLogP(mol), 2),
                'hbd': Descriptors.NumHDonors(mol),
                'hba': Descriptors.NumHAcceptors(mol),
                'tpsa': round(Descriptors.TPSA(mol), 2),
                'rotatable_bonds': Descriptors.NumRotatableBonds(mol),
                'aromatic_rings': Descriptors.NumAromaticRings(mol),
                'heavy_atoms': Descriptors.HeavyAtomCount(mol),
                'formal_charge': Chem.rdmolops.GetFormalCharge(mol),
                'lipinski_violations': self.count_lipinski_violations(mol)
            }
            return properties
        except Exception as e:
            print(f"Error calculando propiedades para {smiles}: {e}")
            return None
    
    def count_lipinski_violations(self, mol):
        """Cuenta violaciones a la regla de Lipinski"""
        violations = 0
        mw = Descriptors.MolWt(mol)
        logp = Descriptors.MolLogP(mol)
        hbd = Descriptors.NumHDonors(mol)
        hba = Descriptors.NumHAcceptors(mol)
        
        if mw > 500: violations += 1
        if logp > 5: violations += 1
        if hbd > 5: violations += 1
        if hba > 10: violations += 1
        
        return violations
    
    def generate_molecule_image(self, smiles, size=(400, 400)):
        """
        Genera imagen de la estructura molecular usando RDKit
        
        Args:
            smiles (str): Código SMILES
            size (tuple): Tamaño de la imagen
            
        Returns:
            PIL.Image: Imagen de la molécula
        """
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                return None
            
            # Generar coordenadas 2D
            from rdkit.Chem import rdDepictor
            rdDepictor.Compute2DCoords(mol)
            
            # Crear imagen
            img = Draw.MolToImage(mol, size=size, kekulize=True)
            return img
        except Exception as e:
            print(f"Error generando imagen para {smiles}: {e}")
            return None
    
    def save_image_for_docx(self, img, filename):
        """
        Guarda imagen en formato compatible con docx
        
        Args:
            img (PIL.Image): Imagen a guardar
            filename (str): Nombre del archivo
            
        Returns:
            str: Ruta del archivo guardado
        """
        if img is None:
            return None
            
        img_path = os.path.join(self.output_dir, f"{filename}.png")
        img.save(img_path, "PNG")
        return img_path
    
    def add_university_logo(self, doc, logo_path, width=Inches(1.5)):
        """
        Añade el logo de la universidad al documento
        
        Args:
            doc: Documento de Word
            logo_path (str): Ruta al archivo del logo
            width: Ancho del logo (por defecto 1.5 pulgadas)
        """
        try:
            if os.path.exists(logo_path):
                # Crear párrafo para el logo
                logo_paragraph = doc.add_paragraph()
                logo_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
                
                # Añadir imagen del logo
                run = logo_paragraph.add_run()
                run.add_picture(logo_path, width=width)
                
                # Añadir espacio después del logo
                doc.add_paragraph()
                
                #print(f"Logo añadido: {logo_path}")
            else:
                print(f"Advertencia: No se encontró el archivo de logo: {logo_path}")
                # Añadir párrafo vacío para mantener el espaciado
                doc.add_paragraph()
        except Exception as e:
            print(f"Error al añadir logo: {e}")
            # Añadir párrafo vacío para mantener el espaciado
            doc.add_paragraph()

    def add_header_footer(self, doc):
        """Añade encabezado y pie de página al documento"""
        # Encabezado
        header = doc.sections[0].header
        header_para = header.paragraphs[0]
        header_para.text = "Informe de Análisis - Blood-Brain Barrier Penetration (BBBP)"
        header_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        # Pie de página
        footer = doc.sections[0].footer
        footer_para = footer.paragraphs[0]
        footer_para.text = f"Generado el {datetime.now().strftime('%d/%m/%Y %H:%M:%S')} por Valeria Candia"
        footer_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    def create_compound_report(self, row, index):
        """
        Crea un informe detallado para un compuesto específico
        
        Args:
            row (pd.Series): Fila del DataFrame con datos del compuesto
            index (int): Índice del compuesto
        """
        # Crear documento
        doc = Document()
        
        # Añadir encabezado y pie de página
        self.add_header_footer(doc)
        
        # --- Añadir logo de la universidad ---
        logo_path = "Parcial2_Candia_Zarco_Valeria_Ej2_img2.png"  # Cambia esta ruta por la correcta
        self.add_university_logo(doc, logo_path, width=Inches(1.5))  # Ajusta el ancho según necesites
        
        # Título principal
        title = doc.add_heading(f'Informe de Compuesto #{index + 1}', 0)
        title.alignment = WD_ALIGN_PARAGRAPH.CENTER
        # Cambiar color del título a negro
        for run in title.runs:
            run.font.color.rgb = RGBColor(0, 0, 0)
        
        # Información básica
        self.add_black_heading(doc, '1. Información General', level=1)
        
        info_table = doc.add_table(rows=4, cols=2)
        info_table.style = 'Table Grid'
        
        info_data = [
            ['Nombre del Compuesto', str(row['name']) if pd.notna(row['name']) else 'No disponible'],
            ['Código SMILES', str(row['smiles'])],
            ['Permeabilidad BBB', 'Penetra' if row['p_np'] == 1 else 'No penetra'],
            ['Clasificación', 'Compuesto permeable' if row['p_np'] == 1 else 'Compuesto no permeable']
        ]
        
        for i, (key, value) in enumerate(info_data):
            info_table.cell(i, 0).text = key
            info_table.cell(i, 1).text = value
            # Hacer la primera columna en negrita
            info_table.cell(i, 0).paragraphs[0].runs[0].font.bold = True
        
        # Generar y añadir imagen de la estructura molecular
        self.add_black_heading(doc, '2. Estructura Molecular', level=1)
        
        img = self.generate_molecule_image(row['smiles'])
        if img:
            img_path = self.save_image_for_docx(img, f"compound_{index}")
            if img_path:
                paragraph = doc.add_paragraph()
                paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
                run = paragraph.runs[0] if paragraph.runs else paragraph.add_run()
                run.add_picture(img_path, width=Inches(4))
                
                # Añadir caption
                caption = doc.add_paragraph(f'Figura 1: Estructura molecular de {row["name"] if pd.notna(row["name"]) else "Compuesto"}')
                caption.alignment = WD_ALIGN_PARAGRAPH.CENTER
                caption.italic = True
        else:
            doc.add_paragraph("No se pudo generar la imagen de la estructura molecular.")
        
        # Propiedades moleculares
        self.add_black_heading(doc, '3. Propiedades Fisicoquímicas', level=1)
        
        properties = self.calculate_molecular_properties(row['smiles'])
        if properties:
            prop_table = doc.add_table(rows=len(properties), cols=2)
            prop_table.style = 'Table Grid'
            
            prop_descriptions = {
                'molecular_weight': 'Peso Molecular (Da)',
                'logp': 'LogP (Coeficiente de partición)',
                'hbd': 'Donadores de enlaces de hidrógeno',
                'hba': 'Aceptores de enlaces de hidrógeno',
                'tpsa': 'Área superficial polar topológica (Ų)',
                'rotatable_bonds': 'Enlaces rotables',
                'aromatic_rings': 'Anillos aromáticos',
                'heavy_atoms': 'Átomos pesados',
                'formal_charge': 'Carga formal',
                'lipinski_violations': 'Violaciones regla de Lipinski'
            }
            
            for i, (key, value) in enumerate(properties.items()):
                prop_table.cell(i, 0).text = prop_descriptions.get(key, key)
                prop_table.cell(i, 1).text = str(value)
                prop_table.cell(i, 0).paragraphs[0].runs[0].font.bold = True
        else:
            # Mensaje cuando no se pueden calcular las propiedades
            no_props_para = doc.add_paragraph()
            no_props_run = no_props_para.add_run("⚠️ No se pudieron calcular las propiedades fisicoquímicas para este compuesto.")
            no_props_run.font.color.rgb = RGBColor(255, 140, 0)  # Color naranja para advertencia
            no_props_run.font.bold = True
            
            doc.add_paragraph("Esto puede deberse a:")
            reasons_para = doc.add_paragraph()
            reasons_para.add_run("• Estructura molecular inválida o incompleta en el código SMILES\n")
            reasons_para.add_run("• Problemas en el procesamiento con RDKit\n")
            reasons_para.add_run("• Caracteres especiales o formato incorrecto en la estructura química")
            
            doc.add_paragraph("Se recomienda verificar la validez del código SMILES y la estructura molecular.")
        
        # Análisis de permeabilidad BBB
        self.add_black_heading(doc, '4. Análisis de Permeabilidad de Barrera Hematoencefálica', level=1)
        
        permeability_status = "penetra" if row['p_np'] == 1 else "no penetra"
        doc.add_paragraph(f"Este compuesto ha sido clasificado como uno que {permeability_status} la barrera hematoencefálica.")
        
        if properties:
            # Análisis basado en propiedades
            analysis_text = self.generate_permeability_analysis(properties, row['p_np'])
            doc.add_paragraph(analysis_text)
        else:
            # Mensaje cuando no hay propiedades disponibles para el análisis
            no_analysis_para = doc.add_paragraph()
            no_analysis_run = no_analysis_para.add_run("ℹ️ El análisis detallado de permeabilidad no está disponible debido a la falta de propiedades fisicoquímicas calculables.")
            no_analysis_run.font.color.rgb = RGBColor(70, 130, 180)  # Color azul para información
            no_analysis_run.font.italic = True
            
            doc.add_paragraph(f"Sin embargo, la clasificación experimental indica que este compuesto {permeability_status} la barrera hematoencefálica según los datos del dataset BBBP.")
        
        # Regla de Lipinski
        self.add_black_heading(doc, '5. Evaluación según Regla de Lipinski', level=1)
        
        if properties and properties['lipinski_violations'] is not None:
            lipinski_text = f"Este compuesto presenta {properties['lipinski_violations']} violación(es) a la regla de Lipinski de los cinco. "
            if properties['lipinski_violations'] == 0:
                lipinski_text += "Esto sugiere buenas propiedades de tipo fármaco y biodisponibilidad oral potencial."
            elif properties['lipinski_violations'] <= 1:
                lipinski_text += "Esto aún sugiere propiedades farmacológicas aceptables."
            else:
                lipinski_text += "Esto puede indicar problemas potenciales con la biodisponibilidad oral."
            
            doc.add_paragraph(lipinski_text)
        else:
            # Mensaje cuando no se puede evaluar la regla de Lipinski
            no_lipinski_para = doc.add_paragraph()
            no_lipinski_run = no_lipinski_para.add_run("⚠️ No se pudo realizar la evaluación según la regla de Lipinski.")
            no_lipinski_run.font.color.rgb = RGBColor(255, 140, 0)  # Color naranja para advertencia
            no_lipinski_run.font.bold = True
            
            doc.add_paragraph("La regla de Lipinski requiere el cálculo de las siguientes propiedades moleculares:")
            lipinski_requirements = doc.add_paragraph()
            lipinski_requirements.add_run("• Peso molecular ≤ 500 Da\n")
            lipinski_requirements.add_run("• LogP ≤ 5\n")
            lipinski_requirements.add_run("• Donadores de enlaces de hidrógeno ≤ 5\n")
            lipinski_requirements.add_run("• Aceptores de enlaces de hidrógeno ≤ 10")
            
            doc.add_paragraph("Sin estas propiedades calculables, no es posible determinar la 'drug-likeness' del compuesto según esta regla.")
        
        # Conclusiones
        self.add_black_heading(doc, '6. Conclusiones', level=1)
        
        conclusion = self.generate_conclusion(row, properties)
        doc.add_paragraph(conclusion)
        
        # Guardar documento
        filename = f"informe_compuesto_{index + 1:03d}_{row['name'] if pd.notna(row['name']) else 'unnamed'}.docx"
        # Limpiar caracteres no válidos del nombre de archivo
        filename = "".join(c for c in filename if c.isalnum() or c in (' ', '-', '_', '.')).rstrip()
        filepath = os.path.join(self.output_dir, filename)
        
        doc.save(filepath)
        print(f"Informe guardado: {filename}")
        
        # Limpiar imagen temporal si existe
        if img:
            temp_img_path = os.path.join(self.output_dir, f"compound_{index}.png")
            if os.path.exists(temp_img_path):
                os.remove(temp_img_path)
    
    def generate_permeability_analysis(self, properties, p_np):
        """Genera análisis de permeabilidad basado en propiedades"""
        analysis = []
        
        # Análisis de peso molecular
        if properties['molecular_weight'] < 400:
            analysis.append("El bajo peso molecular favorece la permeabilidad.")
        elif properties['molecular_weight'] > 600:
            analysis.append("El alto peso molecular puede limitar la permeabilidad.")
        
        # Análisis de LogP
        if 1 <= properties['logp'] <= 3:
            analysis.append("El valor de LogP está en el rango óptimo para permeabilidad BBB.")
        elif properties['logp'] < 1:
            analysis.append("El bajo LogP puede indicar alta polaridad, limitando la permeabilidad.")
        elif properties['logp'] > 3:
            analysis.append("El alto LogP sugiere alta lipofilia, lo que puede favorecer la permeabilidad.")
        
        # Análisis de TPSA
        if properties['tpsa'] < 60:
            analysis.append("La baja área superficial polar favorece la penetración de la BBB.")
        elif properties['tpsa'] > 90:
            analysis.append("La alta área superficial polar puede dificultar la penetración de la BBB.")
        
        if not analysis:
            analysis.append("Las propiedades moleculares muestran características mixtas para la permeabilidad BBB.")
        
        result_text = " ".join(analysis)
        
        # Añadir concordancia con la clasificación
        expected = "favorable" if any("favorece" in a or "óptimo" in a for a in analysis) else "desfavorable"
        actual = "penetra" if p_np == 1 else "no penetra"
        
        if (expected == "favorable" and p_np == 1) or (expected == "desfavorable" and p_np == 0):
            result_text += f" Esta evaluación es consistente con la clasificación experimental de que el compuesto {actual} la barrera."
        else:
            result_text += f" Interesantemente, a pesar de las características aparentemente {expected}, el compuesto {actual} la barrera según datos experimentales."
        
        return result_text
    
    def generate_conclusion(self, row, properties):
        """Genera conclusión del informe"""
        compound_name = row['name'] if pd.notna(row['name']) else "Este compuesto"
        permeability = "penetra" if row['p_np'] == 1 else "no penetra"
        
        conclusion = f"{compound_name} ha sido clasificado experimentalmente como un compuesto que {permeability} la barrera hematoencefálica. "
        
        if properties:
            if properties['lipinski_violations'] <= 1 and row['p_np'] == 1:
                conclusion += "Sus propiedades fisicoquímicas son generalmente favorables tanto para la permeabilidad BBB como para características de tipo fármaco. "
            elif properties['lipinski_violations'] > 1:
                conclusion += "Presenta algunas características que pueden limitar su potencial como fármaco oral. "
            
            conclusion += f"Con un peso molecular de {properties['molecular_weight']} Da y un LogP de {properties['logp']}, "
            
            if row['p_np'] == 1:
                conclusion += "este compuesto representa un candidato interesante para el desarrollo de fármacos dirigidos al sistema nervioso central."
            else:
                conclusion += "este compuesto podría requerir modificaciones estructurales para mejorar su permeabilidad si se desea actividad en el SNC."
        
        return conclusion
    
    def generate_reports(self, num_reports=200):
        """
        Genera informes para los primeros num_reports compuestos
        
        Args:
            num_reports (int): Número de informes a generar (máximo 250)
        """
        # Validar que no exceda el límite máximo
        max_allowed = min(250, len(self.df))
        if num_reports > max_allowed:
            print(f"⚠️  Número solicitado ({num_reports}) excede el máximo permitido ({max_allowed})")
            num_reports = max_allowed
            print(f"📝 Se generarán {num_reports} informes en su lugar.")
        
        print(f"\n📊 Generando {num_reports} informes...")
        print("=" * 50)
        
        # Limitar al número disponible de compuestos
        num_reports = min(num_reports, len(self.df))
        
        # Variables para tracking de progreso
        successful_reports = 0
        failed_reports = 0
        
        for i in range(num_reports):
            try:
                self.create_compound_report(self.df.iloc[i], i)
                successful_reports += 1
                
                # Mostrar progreso cada 5 informes o al final
                if (i + 1) % 5 == 0 or (i + 1) == num_reports:
                    progress_percent = ((i + 1) / num_reports) * 100
                    print(f"📈 Progreso: {i + 1}/{num_reports} informes completados ({progress_percent:.1f}%)")
                    
            except Exception as e:
                failed_reports += 1
                print(f"❌ Error generando informe {i + 1}: {e}")
                continue
        
        # Resumen final
        print("\n" + "=" * 50)
        print("📋 RESUMEN DE GENERACIÓN:")
        print(f"✅ Informes exitosos: {successful_reports}")
        if failed_reports > 0:
            print(f"❌ Informes fallidos: {failed_reports}")
        print(f"📁 Archivos guardados en: {self.output_dir}")
        print("=" * 50)
        print("✨ ¡Proceso completado!")
        
        return successful_reports, failed_reports

# Función principal para usar el generador
def main():
    """
    Función principal para ejecutar el generador de informes
    """
    # Ruta al archivo CSV
    csv_path = "BBBP.csv"
    
    # Verificar que el archivo existe
    if not os.path.exists(csv_path):
        print(f"Error: No se encontró el archivo {csv_path}")
        print("Por favor, ajusta la ruta del archivo CSV en la variable csv_path")
        return
    
    # Crear generador de informes
    generator = BBBPReportGenerator(csv_path)
    
    # Obtener número de informes del usuario
    print("=== GENERADOR DE INFORMES BBBP ===")
    print(f"Archivo CSV cargado: {csv_path}")
    print(f"Total de compuestos disponibles: {len(generator.df)}")
    print("\n¿Cuántos informes deseas generar?")
    
    while True:
        try:
            user_input = input(f"Ingresa un número entre 1 y 250 (máximo disponible: {min(250, len(generator.df))}): ")
            
            # Verificar si el usuario quiere salir
            if user_input.lower() in ['salir', 'exit', 'quit', 'q']:
                print("Operación cancelada.")
                return
            
            num_reports = int(user_input)
            
            # Validar el rango
            max_allowed = min(250, len(generator.df))
            if num_reports < 1:
                print("❌ Error: El número debe ser mayor a 0.")
                continue
            elif num_reports > max_allowed:
                print(f"❌ Error: El número máximo permitido es {max_allowed}.")
                continue
            else:
                break
                
        except ValueError:
            print("❌ Error: Por favor ingresa un número válido.")
            continue
    
    # Confirmar la operación
    print(f"\n📋 Se generarán {num_reports} informes.")
    confirmacion = input("¿Continuar? (s/n): ").lower().strip()
    
    if confirmacion in ['s', 'si', 'sí', 'y', 'yes']:
        print(f"\n🚀 Iniciando generación de {num_reports} informes...")
        generator.generate_reports(num_reports=num_reports)
    else:
        print("Operación cancelada.")
        return

if __name__ == "__main__":
    main()

=== GENERADOR DE INFORMES BBBP ===
Archivo CSV cargado: BBBP.csv
Total de compuestos disponibles: 2050

¿Cuántos informes deseas generar?

📋 Se generarán 200 informes.

🚀 Iniciando generación de 200 informes...

📊 Generando 200 informes...
Informe guardado: informe_compuesto_001_Propanolol.docx
Informe guardado: informe_compuesto_002_Terbutylchlorambucil.docx
Informe guardado: informe_compuesto_003_40730.docx
Informe guardado: informe_compuesto_004_24.docx
Informe guardado: informe_compuesto_005_cloxacillin.docx
📈 Progreso: 5/200 informes completados (2.5%)
Informe guardado: informe_compuesto_006_cefoperazone.docx
Informe guardado: informe_compuesto_007_rolitetracycline.docx
Informe guardado: informe_compuesto_008_ondansetron.docx
Informe guardado: informe_compuesto_009_diltiazem.docx
Informe guardado: informe_compuesto_010_Amiloride.docx
📈 Progreso: 10/200 informes completados (5.0%)
Informe guardado: informe_compuesto_011_M2L-663581.docx
Informe guardado: informe_compuesto_012_alovud

[20:55:31] Explicit valence for atom # 1 N, 4, is greater than permitted
[20:55:31] Explicit valence for atom # 1 N, 4, is greater than permitted
[20:55:31] WARNING: not removing hydrogen atom without neighbors
[20:55:31] WARNING: not removing hydrogen atom without neighbors
[20:55:31] Explicit valence for atom # 6 N, 4, is greater than permitted
[20:55:31] Explicit valence for atom # 6 N, 4, is greater than permitted


Informe guardado: informe_compuesto_062_22767.docx
Informe guardado: informe_compuesto_063_vincristine.docx
Informe guardado: informe_compuesto_064_SB-222200.docx
Informe guardado: informe_compuesto_065_SB-656104-A.docx
📈 Progreso: 65/200 informes completados (32.5%)
Informe guardado: informe_compuesto_066_compound 38.docx
Informe guardado: informe_compuesto_067_maprotiline.docx
Informe guardado: informe_compuesto_068_compound 43.docx
Informe guardado: informe_compuesto_069_MIL-663581.docx
Informe guardado: informe_compuesto_070_cefotaxime.docx
📈 Progreso: 70/200 informes completados (35.0%)
Informe guardado: informe_compuesto_071_carbamazepine-10.docx
Informe guardado: informe_compuesto_072_Cyclohexane.docx
Informe guardado: informe_compuesto_073_2.docx
Informe guardado: informe_compuesto_074_SKF89124.docx
Informe guardado: informe_compuesto_075_Org5222.docx
📈 Progreso: 75/200 informes completados (37.5%)
Informe guardado: informe_compuesto_076_compound 35.docx
Informe guardado: infor

[20:55:40] WARNING: not removing hydrogen atom without neighbors
[20:55:40] WARNING: not removing hydrogen atom without neighbors


Informe guardado: informe_compuesto_129_Betamethasone.docx
Informe guardado: informe_compuesto_130_carbenicillin.docx
📈 Progreso: 130/200 informes completados (65.0%)
Informe guardado: informe_compuesto_131_carbidopa.docx
Informe guardado: informe_compuesto_132_carteolol.docx
Informe guardado: informe_compuesto_133_Cefazolin.docx
Informe guardado: informe_compuesto_134_cephapirin.docx
Informe guardado: informe_compuesto_135_chlorambucil.docx
📈 Progreso: 135/200 informes completados (67.5%)
Informe guardado: informe_compuesto_136_pergolide.docx
Informe guardado: informe_compuesto_137_perlapine.docx
Informe guardado: informe_compuesto_138_perphenazine-HCl.docx
Informe guardado: informe_compuesto_139_pethidine.docx
Informe guardado: informe_compuesto_140_phencyclidine.docx
📈 Progreso: 140/200 informes completados (70.0%)
Informe guardado: informe_compuesto_141_phenelzine sulfate.docx
Informe guardado: informe_compuesto_142_phenobarbital.docx
Informe guardado: informe_compuesto_143_phenpro

[20:55:48] WARNING: not removing hydrogen atom without neighbors
[20:55:48] WARNING: not removing hydrogen atom without neighbors


Informe guardado: informe_compuesto_197_spiclomazine.docx
Informe guardado: informe_compuesto_198_Atovaquone.docx
Informe guardado: informe_compuesto_199_Azelaicacid.docx
Informe guardado: informe_compuesto_200_Azithromycin.docx
📈 Progreso: 200/200 informes completados (100.0%)

📋 RESUMEN DE GENERACIÓN:
✅ Informes exitosos: 200
📁 Archivos guardados en: informes_bbbp
✨ ¡Proceso completado!
